In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes

# Gradient Descent

Gradient descent is a method for unconstrained mathematical **optimization**. It is a first-order iterative algorithm for finding a **local minimum** of a differentiable multivariate function. It is particularly useful in machine learning for minimizing the cost or loss function.

The idea is to take **repeated steps in the opposite direction of the gradient** (or approximate gradient) of the function at the current point, because this is the direction of steepest descent. Conversely, stepping in the direction of the gradient will lead to a local maximum of that function; the procedure is then known as gradient ascent.

![](https://easyai.tech/wp-content/uploads/2019/01/tiduxiajiang-1.png)

## Intuition

Gradient descent operates on the principle of iteratively refining model parameters to minimize the cost function, drawing an analogy to navigating terrain to find the lowest point. In this analogy, the cost function represents the landscape's elevation, and the model parameters correspond to the current location. Initially, the algorithm starts at a random point on the landscape and senses the slope beneath its feet. By computing the gradient of the cost function, it determines the direction of the steepest descent. Each step taken in this direction adjusts the parameters, gradually descending towards the valley floor, where the cost function is minimized. Through repeated iterations, the algorithm refines the parameters until it reaches a point where the slope is nearly flat, signifying convergence to a local or global minimum. Ultimately, gradient descent guides the model towards optimal parameter values, effectively minimizing the cost function and improving model performance.

$$\hat{y_i} = w_1x_1 + w_2x_2 + \dots + w_nx_n + b$$

$$\text{Loss Function: J (w,b)} = \sum_{i=1}^D (\hat{y_i}-y_i)^2 = \sum_{i=1}^D ({w_1x_1 + w_2x_2 + \dots + w_nx_n + b}-y_i)^2$$

$$\frac{\partial{J}}{\partial{w}} = 2  $$  
$$\frac{\partial{J}}{\partial{b}}$$

## Algorithm of Gradient Descent

1. Initialize model parameters randomly or with predefined values.
2. Compute the gradient of the cost function with respect to the parameters using all training examples.
3. Update parameters by subtracting the product of the learning rate and the gradient.
4. Repeat steps 2 and 3 until convergence or a maximum number of iterations is reached.

## Types of Gradient Descent

1. Batch Gradient Descent
2. Stochastic Gradient Descent
3. Mini Batch Gradient Descent

### Batch Gradient Descent

It updates model parameters using gradients computed over the entire training dataset in each iteration. It offers stable convergence but can be computationally expensive, especially for large datasets.

In [106]:
X,y = load_diabetes(return_X_y=True)

In [107]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2002)

In [108]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train,y_train)

LinearRegression()

In [109]:
lr.score(X_test,y_test)

0.49731505640194407

In [110]:
lr.coef_, lr.intercept_

(array([  -47.92919504,  -335.02937809,   510.28947017,   350.81248009,
        -1299.57463645,   953.75290824,   249.29502423,   173.9486256 ,
          929.41771914,    54.59466589]),
 148.832300854267)

#### Own Implementation

In [111]:
class BGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            # update all the coef and the intercept
            y_hat = np.dot(X_train,self.coef_) + self.intercept_
            intercept_der = -2 * np.mean(y_train - y_hat)
            self.intercept_ = self.intercept_ - (self.lr * intercept_der)
            
            coef_der = -2 * np.dot((y_train - y_hat),X_train)/X_train.shape[0]
            self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_
    

In [113]:
bgdr = BGDRegressor(epochs=1000,learning_rate=0.5)
bgdr.fit(X_train,y_train)

149.46680363175685 [ -29.62178873 -266.48595849  488.51570984  311.99880642  -31.21243041
  -79.58505929 -211.99496796  124.50554176  412.00252497  110.00301171]


In [114]:
from sklearn.metrics import r2_score

y_pred = bgdr.predict(X_test)
r2_score(y_test,y_pred)

0.5225100226742716

### Stochastic Gradient Descent

It updates model parameters using the gradient of the cost function with respect to a single training example at a time. It's efficient for large datasets, converges faster, and is robust to local minima, but can exhibit noisy convergence due to single-example updates.

In [88]:
X,y = load_diabetes(return_X_y=True)

In [93]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2002)

In [94]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train,y_train)

LinearRegression()

In [95]:
lr.score(X_test,y_test)

0.49731505640194407

In [96]:
lr.coef_, lr.intercept_

(array([  -47.92919504,  -335.02937809,   510.28947017,   350.81248009,
        -1299.57463645,   953.75290824,   249.29502423,   173.9486256 ,
          929.41771914,    54.59466589]),
 148.832300854267)

#### Own Implementation

In [97]:
class SGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)
                
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [98]:
sgd = SGDRegressor(learning_rate=0.01,epochs=40)

In [99]:
import time

start = time.time()
sgd.fit(X_train,y_train)
print("The time taken is",time.time() - start)

146.1409855015611 [  20.329438    -86.5950672   284.22881403  198.68588478   49.82538328
   22.44773751 -143.42933769  132.06268911  250.21045523  125.15945141]
The time taken is 0.4344296455383301


In [101]:
from sklearn.metrics import r2_score

y_pred = sgd.predict(X_test)
r2_score(y_test,y_pred)

0.454568491032212

#### `scikit-learn` Implementation

In [103]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(max_iter=100,learning_rate='constant',eta0=0.01)
sgd.fit(X_train,y_train)

C:\Users\Danish\anaconda3\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(learning_rate='constant', max_iter=100)

In [104]:
sgd.score(X_test,y_test)

0.4869186586701415

In [105]:
sgd.coef_, sgd.intercept_

(array([  16.70225531, -109.81478296,  320.24001347,  216.29628752,
          38.59398519,    7.7477704 , -158.72759047,  136.87665245,
         273.53568522,  129.82723449]),
 array([149.58456529]))

### Mini Batch Gradient Descent

It is a method used to train machine learning models by updating parameters based on small subsets of the training data instead of the entire dataset. It combines the efficiency of batch gradient descent with the stochastic nature of stochastic gradient descent. This approach is computationally efficient, introduces regularization, and often leads to smoother convergence during training.

In [3]:
X, y = load_diabetes(return_X_y=True)

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2002)

In [5]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train,y_train)

LinearRegression()

In [6]:
lr.score(X_test,y_test)

0.49731505640194407

In [7]:
lr.coef_, lr.intercept_

(array([  -47.92919504,  -335.02937809,   510.28947017,   350.81248009,
        -1299.57463645,   953.75290824,   249.29502423,   173.9486256 ,
          929.41771914,    54.59466589]),
 148.832300854267)

#### Own Implementation

In [8]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_
    

In [9]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)
mbr.fit(X_train,y_train)

y_pred = mbr.predict(X_test)

150.28341259895402 [-1.05227941e+01 -1.99333949e+02  4.30803822e+02  2.85844475e+02
  4.29967973e-01 -4.20797872e+01 -1.96280926e+02  1.37108720e+02
  3.58663858e+02  1.28250832e+02]


In [10]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.5230814384866878

#### `scikit-learn` Implementation

In [84]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='adaptive', eta0=0.15,random_state=2002)

In [85]:
batch_size = 35

for i in range(100):
    
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [86]:
sgd.score(X_test,y_test)

0.5095359725795132

In [87]:
sgd.coef_, sgd.intercept_

(array([ -36.50045865, -179.28911745,  408.47005314,  250.31918859,
          17.79855534,  -36.48915894, -173.66418293,  141.08146966,
         348.14450429,  114.90907008]),
 array([149.19500524]))

### Batch vs Stochastic vs Mini-Batch Gradient Descent

![](https://miro.medium.com/v2/resize:fit:908/1*bKSddSmLDaYszWllvQ3Z6A.png)